In [552]:
import random

In [553]:
wuerfel = []
wuerfel.append([random.randint(1, 6) for i in range(0, 5)])
wuerfel=wuerfel[0]
print(sorted(wuerfel))

[1, 2, 2, 3, 5]


In [554]:
classCheck = [0, 0, 0, 0, 0, 0, 0, 0]
classes = (
    "P",  # Augenzahl
    "3P",  # 3er Pasch
    "4P",  # 4er Pasch
    "F",  # Full House
    "KS",  # Kleine Straße
    "GS",  # Große Straße
    "K",  # Kniffel
    "C",  # Chance
)
classDict = dict(zip(classes, classCheck))
print(sorted(wuerfel))

[1, 2, 2, 3, 5]


In [555]:
wuerfel.sort()
paare=[]
for l in range(1, 7):

    for i in range(0, 5):  # 1. Stelle i

        for j in range(0, 5):  # 2. Stelle j
            if j != i:

                for k in range(0, 5):  # 3. Stelle k
                    if k != j and k != i:

                        # 3er Pasch
                        if (
                            wuerfel[i] == wuerfel[j] == wuerfel[k]
                            and classDict["3P"] == 0
                        ):
                            classCheck[1] = wuerfel[i]
                            classDict["3P"] = 1
                            print(f"3er Pasch mit Zahl: {wuerfel[i]}")

                        # Paare für Fullhouse schauen
                        if (
                            wuerfel[i] == wuerfel[j]
                            and classDict["P"] < 2
                            and classCheck[0] != wuerfel[i]
                            and classCheck[1] != wuerfel[i]
                            and l > 1
                        ):
                            classCheck[0] = wuerfel[i]
                            paare.append(wuerfel[i])
                            classDict["P"] += 1
                            print(f"Paar mit Zahl: {wuerfel[i]}")

                        for m in range(0, 5):  # 4. Stelle m

                            if m != j and m != i and m != k:
                                pass
                        #         for n in range(0, 5):   # 5. Stelle n
                        #             if n != j and n != i and n != k and n!=m:
                        #                 pass

print("Paare:",paare)

Paar mit Zahl: 2
Paare: [2]
